# Анализ Данных
## Лабораторная работа №1. Нахождение похожих предложений в тексте

Выполнил: *Кулеш Антон* <br />

*Минск, осень 2016*

----
### Постановка задачи
Для [данного текста](http://www.gutenberg.org/cache/epub/1661/pg1661.txt) построить векторное представление и исследовать связи между предложениями.

In [1]:
import re
import time
from urllib.request import urlopen
from collections import Counter

from scipy import spatial
import numpy as np

### Загрузка и обработка данных

In [2]:
holmes_url = "http://www.gutenberg.org/cache/epub/1661/pg1661.txt"
response = urlopen(holmes_url)
text = response.read().decode('utf8')

### Токенизация
Все слова приводим к нижнему регистру, далее разбиваем текст по всем небуквенным символам.

In [3]:
text_lowercase = text.lower()

In [4]:
tokens = re.split('[^a-z]', text_lowercase)
print(len(tokens))

147772


### Фильтрация
Удаляем слова нулевой длины (пустые строки).

In [5]:
filt_tokens = list(filter(lambda x: len(x) >= 1, tokens))
print(len(filt_tokens))

109000


### Словник
Создадим словарь, в котором будем хранить все слова, которые встречаются в тексте и их частоты.

In [6]:
vocabulary = Counter(filt_tokens)
len(vocabulary)

8068

### Разбиение текста
Выполним разбиение данного текста на предложения по шаблону **[\!\?\.!]**.

In [34]:
def splitting(text):
    p = re.compile(r'[\?\.\!]')
    sentences = re.split(p, text_lowercase)
    '''
    import nltk
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sent_detector.tokenize(text_lowercase.strip())
    '''
    sentences = [re.split('[^a-z]', sentence) for sentence in sentences]
    sentences = [list(filter(lambda x: len(x) >= 1, sentence)) for sentence in sentences]
    sentences = [sentence for sentence in sentences if len(sentence) > 1]
    return sentences

In [35]:
sentences = splitting(text_lowercase)
print(len(sentences))

7061


### Векторное представление
Напишем функцию, которая заданному предложению будет ставить в соответствие вектор частот, размерность которого соответствует размеру словника.

In [11]:
def sent2vec(sentence, *, vocabulary):
    "Vector representing of sentence"
    n = len(vocabulary)
    local_vocabulary = Counter(sentence)
    vector = np.zeros(n)
    for i, word in enumerate(vocabulary.keys()):
        if word in local_vocabulary:
            vector[i] = local_vocabulary[word]
    return vector

Таким образом исходный текст можно представить в виде [терм-документной матрицы](https://ru.wikipedia.org/wiki/Терм-документная_матрица).

In [12]:
term_matrix = np.asarray(list(map(lambda x: sent2vec(x, vocabulary=vocabulary), sentences)))

In [13]:
term_matrix.shape

(7061, 8068)

### Вычисление сходства
Так как предложение теперь -- вектра в пространстве $R^{8068}$, то между этими векторами можно вычислить расстояние, использую известные меры. В нашей задаче будем использовать [косинусную меру](https://en.wikipedia.org/wiki/Cosine_similarity).

In [14]:
def most_similar(n_sentence, *, term_matrix):
    m, n = term_matrix.shape
    dist = 1
    neighbour = n_sentence
    for i in range(m):
        curr_dist = spatial.distance.cosine(term_matrix[n_sentence],term_matrix[i])
        if curr_dist < dist and i != n_sentence:
            dist = curr_dist
            neighbour = i
    return neighbour, dist

In [33]:
%%time
most_similar(6000, term_matrix=term_matrix)

Wall time: 2.77 s


(4736, 0.44966611633250775)

In [34]:
Counter(sentences[6000])

Counter({'arthur': 1,
         'cousin': 1,
         'crime': 1,
         'feel': 1,
         'i': 2,
         'in': 1,
         'innocent': 1,
         'is': 2,
         'my': 1,
         'of': 1,
         'proving': 1,
         'sir': 1,
         'succeed': 1,
         'sure': 1,
         'that': 2,
         'the': 1,
         'this': 1,
         'trust': 1,
         'truth': 1,
         'what': 1,
         'will': 1,
         'you': 1})

In [35]:
Counter(sentences[4736])

Counter({'beg': 1,
         'business': 1,
         'i': 2,
         'is': 1,
         'my': 1,
         'of': 1,
         'said': 1,
         'sir': 1,
         'state': 1,
         'that': 1,
         'time': 1,
         'value': 1,
         'will': 1,
         'you': 1,
         'your': 1})

Можно отметить, что на схожесть предложений сильно влияют [стоп-слова](https://en.wikipedia.org/wiki/Stop_words), поэтому часто при построении векторных представлений текстовых документов эти "шумовые" слова игнорируют.